In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as :

    SETTING = yaml.load()

PATH = make_path_dict(SETTING)

In [ ]:
enst_tpms = []

for sample_id, fastqs in SETTING["fastq_id_fastq_file_paths"].items():

    enst_tpm = pd.read_csv(
        os.path.join(PATH["kallisto/"], sample_id, "abundance.tsv"),
        sep="\t",
        index_col=0,
    )["tpm"]

    enst_tpm.name = sample_id

    enst_tpms.append(enst_tpm)

enst_x_sample = pd.concat(enst_tpms, axis=1)

enst_x_sample.index.name = "ENST"

enst_x_sample.to_csv(PATH["enst_x_sample.tsv"], sep="\t")

enst_x_sample

In [ ]:
enst_gene_name = pd.read_csv(SETTING["enst_gene_name_file_path"], sep="\t")

gene_x_sample = pd.DataFrame(
    enst_x_sample,
    index=enst_x_sample.index.map(
        {
            enst: gene_name.str.upper()
            for enst, gene_name in zip(
                enst_gene_name["Transcript stable ID version"],
                enst_gene_name["Gene name"],
            )
        }
    ),
)

gene_x_sample.loc[~gene_x_sample.index.isna()].groupby(level=0).median()

gene_x_sample.index.name = "Gene"

gene_x_sample.to_csv(PATH["gene_x_sample.tsv"], sep="\t")

gene_x_sample